In [3]:
import pytorch_transformers
import sys
import torch
%load_ext autoreload
%autoreload 2
sys.path.insert(0, "/ssd2/arthur/TREC2019/scripts/")
from msmarco_dataset import MsMarcoDataset
from args_parser import getArgs
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
from pytorch_transformers import AdamW, WarmupLinearSchedule


In [4]:
is_distil

NameError: name 'is_distil' is not defined

In [6]:
data_dir = "/ssd2/arthur/TREC2019/data/"
argv = [
    "--data_dir", data_dir,
    "--train_file", data_dir + "/train-triples.0",
    "--dev_file", data_dir + "/dev-triples.0",
    "--per_gpu_train_batch_size", "8",
    "--train_batch_size", "32",
    "--gradient_accumulation_steps", "10",
    "--ignore_gpu_ids", "0,1,2,5,6",
    "--limit_gpus", "-1",
    "--eval_steps", "10",
    "--bert_model", "distilbert-base-uncased"
]
args = getArgs(argv)


In [7]:
is_distil = "distilbert" in args.bert_model

In [11]:
not is_distil

False

In [3]:
train_dataset = MsMarcoDataset(args.train_file, args.data_dir, invert_label=False, distil=True)
dev_dataset = MsMarcoDataset(args.dev_file, args.data_dir, invert_label=False, distil=True)

None



None


In [4]:
model = pytorch_transformers.DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

In [5]:
train_data_loader = DataLoader(train_dataset, batch_size=args.train_batch_size, shuffle=True)
dev_data_loader = DataLoader(dev_dataset, batch_size = args.eval_batch_size, shuffle=False)

In [6]:
n_gpu = 0
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
    # We DO NOT want to limit the number of GPUs to be used
    if args.limit_gpus < 0:
        args.limit_gpus = torch.cuda.device_count()
        n_gpu = torch.cuda.device_count()
    # We WANT to limit the number of GPUs to be used
    else:
        n_gpu = min(torch.cuda.device_count(), args.limit_gpus)

device = torch.device("cuda" if (torch.cuda.is_available()
                                 and n_gpu > 0 and args.limit_gpus != 1) else "cpu")
if n_gpu > 0:
    gpu_ids = list(range(n_gpu))
    # Ignore any GPU? (usefull if there is more users on current machine, already using a GPU)
    if args.ignore_gpu_ids is not None:
        for _id in args.ignore_gpu_ids:
            if _id in gpu_ids:
                gpu_ids.remove(_id)
    model = torch.nn.DataParallel(model, device_ids=gpu_ids)
if n_gpu > 0:
    device_0 = torch.device("cuda:{}" .format(gpu_ids[0]))
    model.to(device_0)


In [7]:
warmup_proportion=0.1
weight_decay=0.0
no_decay = ['bias', 'LayerNorm.weight']
lr = args.learning_rate
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay)], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
num_train_optimization_steps = len(train_data_loader) // args.gradient_accumulation_steps * args.n_epochs

optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
warmup_steps = num_train_optimization_steps * warmup_proportion
scheduler = WarmupLinearSchedule(
    optimizer, warmup_steps=warmup_steps, t_total=num_train_optimization_steps)


/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:72: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [8]:
tr_loss = 0.0
for step, batch in tqdm(enumerate(train_data_loader), desc="Batches"):
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
          'attention_mask': batch[1],
          'labels': batch[3]}
    model.train()
    outputs = model(**inputs)
    loss = outputs[0]
    if len(loss) > 1:
        loss = loss.mean()
    if args.gradient_accumulation_steps > 1:
        loss = loss / args.gradient_accumulation_steps
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    loss.backward()
    tr_loss += loss.item()
    if (step + 1) % args.gradient_accumulation_steps == 0:
        optimizer.step()
        scheduler.step()
        model.zero_grad()
    

/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [13]:
type(model.module)

pytorch_transformers.modeling_distilbert.DistilBertForSequenceClassification

In [14]:
eval_loss = 0
preds = None
out_label_ids = None
for batch in tqdm(dev_data_loader):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    break
#     with torch.no_grad():
#         if model.module

In [18]:
issubclass(type(model.module), pytorch_transformers.DistilBertModel)

False

In [22]:
with torch.no_grad():
    if isinstance(model.module, pytorch_transformers.DistilBertForSequenceClassification):
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[3]}
    else:
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'next_sentence_label': batch[3]}
    outputs = model(**inputs)

/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [27]:
eval_loss = 0.0
tmp_eval_loss, logits = outputs[:2]
eval_loss += tmp_eval_loss.mean().item()

In [35]:
import numpy as np
from sklearn.metrics import f1_score, average_precision_score, accuracy_score

preds = logits.detach().cpu().numpy()
out_label_ids = inputs['labels'].detach().cpu().numpy().flatten()
preds = np.argmax(preds, axis=1)
assert len(preds) == len(out_label_ids)

result = {}
result["acc"] = accuracy_score(out_label_ids, preds)
result["f1"] = f1_score(out_label_ids, preds)
result["AP"] = average_precision_score(out_label_ids, preds)
result["acc_and_f1"] = (result["acc"] + result["f1"]) / 2

In [36]:
result

{'acc': 0.0078125,
 'f1': 0.015503875968992248,
 'AP': 0.0078125,
 'acc_and_f1': 0.011658187984496124}

In [38]:
preds

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
model.train()
output = model(**inputs)

In [ ]:
loss = output[0]
if len(loss) > 1:
    loss = loss.mean()
    

In [ ]:
loss